In [1]:
import sys, os
%load_ext autoreload
%autoreload 2
import torch
import torch.nn.functional as F
from torch.autograd import Variable
import torch.distributions as distrib
import torch.nn as nn
import torch.utils.data as td
from torch.utils.data import Dataset, DataLoader
import gym
import numpy as np
#%matplotlib notebook
import matplotlib.pyplot as plt
plt.switch_backend('TKAgg') #('Qt5Agg')
from scipy.sparse import coo_matrix
#from envs import Discrete_Pos_Trap, Gym_Env, Print_Env
from torch.distributions.utils import lazy_property
import foundation as fd
import ray
#import sequential_ray as ray

/home/fleeb/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
ray.init(redirect_worker_output=False, redirect_output=False)

Process STDOUT and STDERR is being redirected to /tmp/raylogs/.
Waiting for redis server at 127.0.0.1:58934 to respond...
Waiting for redis server at 127.0.0.1:37367 to respond...
Starting local scheduler with the following resources: {'CPU': 8, 'GPU': 0}.

View the web UI at http://localhost:8889/notebooks/ray_ui76960.ipynb?token=a1903c26eea2ab7a3bf98b4688f755b7800590b3a8b0cf7b



{'local_scheduler_socket_names': ['/tmp/scheduler1734591'],
 'node_ip_address': '127.0.0.1',
 'object_store_addresses': [ObjectStoreAddress(name='/tmp/plasma_store47471166', manager_name='/tmp/plasma_manager21668872', manager_port=13234)],
 'redis_address': '127.0.0.1:58934',
 'webui_url': 'http://localhost:8889/notebooks/ray_ui76960.ipynb?token=a1903c26eea2ab7a3bf98b4688f755b7800590b3a8b0cf7b'}

In [3]:
env = fd.util.get_env('cartpole')
print(env)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer = nn.Linear(10,10)
        
    def forward(self, x):
        return self.layer(x)

In [42]:
t = Net()
t_id = ray.put(t)
print(list(t.layer.parameters()))
print(list(ray.get(t_id).layer.parameters()))

[Parameter containing:
tensor([[-0.2803,  0.1722,  0.1926,  0.0343,  0.2413, -0.1546, -0.0713,
         -0.3091,  0.1581, -0.2792],
        [-0.0791, -0.2604, -0.1234, -0.0609,  0.2831, -0.0431,  0.2172,
         -0.0222,  0.0312,  0.0840],
        [-0.1822, -0.2972,  0.1844,  0.1027,  0.2662, -0.2672,  0.3119,
          0.1271,  0.0159, -0.1915],
        [ 0.2191, -0.2943,  0.1295,  0.0075,  0.0440, -0.0787, -0.2511,
         -0.1577,  0.1129, -0.1511],
        [-0.2874, -0.2944,  0.1503,  0.0745, -0.2493,  0.1051, -0.2933,
         -0.2560, -0.2219,  0.1242],
        [ 0.0267, -0.2038,  0.1651,  0.2291,  0.1258, -0.1417,  0.2241,
          0.1861, -0.3013, -0.2173],
        [-0.1352, -0.2952, -0.1533, -0.1663, -0.1900, -0.2311,  0.0845,
          0.1739, -0.0236,  0.1423],
        [-0.0065, -0.2335, -0.2206, -0.0934, -0.0611,  0.2480, -0.1714,
          0.0322, -0.0525, -0.2977],
        [-0.1030,  0.1067,  0.0006,  0.0393, -0.2684,  0.1351,  0.0434,
         -0.1972, -0.1552, -0.308

In [36]:
@ray.remote
def test(e):
    x = torch.randn(10)
    return e.layer
    return list(e.parameters())
    #return e(x)

In [39]:
print(list(ray.get(test.remote(t)).parameters()))

[Parameter containing:
tensor([]), Parameter containing:
tensor([])]


In [38]:
print(ray.get(test.remote(t)))

Linear(in_features=10, out_features=10, bias=True)


In [19]:
@ray.remote
def test(n):
    return {n[0][1]+4:1}

In [20]:
j = [test.remote([ray.put({1:i})]) for i in range(10)]

Remote function __main__.test failed with:

Traceback (most recent call last):
  File "<ipython-input-19-e0162ab9c08b>", line 3, in test
TypeError: 'common.ObjectID' object is not subscriptable


  You can inspect errors by running

      ray.error_info()

  If this driver is hanging, start a new one with

      ray.init(redis_address="127.0.0.1:59309")
  
Remote function __main__.test failed with:

Traceback (most recent call last):
  File "<ipython-input-19-e0162ab9c08b>", line 3, in test
TypeError: 'common.ObjectID' object is not subscriptable


  You can inspect errors by running

      ray.error_info()

  If this driver is hanging, start a new one with

      ray.init(redis_address="127.0.0.1:59309")
  
Remote function __main__.test failed with:

Traceback (most recent call last):
  File "<ipython-input-19-e0162ab9c08b>", line 3, in test
TypeError: 'common.ObjectID' object is not subscriptable


  You can inspect errors by running

      ray.error_info()

  If this driver is hangi

In [16]:
ray.get(j[0])

{4: 1}

In [17]:
ray.get(j[0])

{4: 1}

In [18]:
@ray.remote
def test2(ls):
    return [l+1 for l in ls]

In [19]:
ray.get(test2.remote(j))

Remote function __main__.test2 failed with:

Traceback (most recent call last):
  File "<ipython-input-18-d6e74c6d8a6f>", line 3, in test2
  File "<ipython-input-18-d6e74c6d8a6f>", line 3, in <listcomp>
TypeError: unsupported operand type(s) for +: 'common.ObjectID' and 'int'


  You can inspect errors by running

      ray.error_info()

  If this driver is hanging, start a new one with

      ray.init(redis_address="127.0.0.1:21234")
  


RayGetError: Could not get objectid ObjectID(e199949e5af0df2f663fc0158dd02027d0ebc5fc). It was created by remote function [31m__main__.test2[39m which failed with:

Remote function [31m__main__.test2[39m failed with:

Traceback (most recent call last):
  File "<ipython-input-18-d6e74c6d8a6f>", line 3, in test2
  File "<ipython-input-18-d6e74c6d8a6f>", line 3, in <listcomp>
TypeError: unsupported operand type(s) for +: 'common.ObjectID' and 'int'


In [16]:
ray.get(test.remote(1))

5

In [7]:
@ray.remote
def test():
    print('worked')

In [8]:
test()

decorator
worked


In [17]:
ray.get(x)

4

In [18]:
t.a

{3: 4}

In [19]:
t.get_a()

Exception: Remote functions cannot be called directly. Instead of running '__main__.get_a()', try '__main__.get_a.remote()'.

In [4]:
@ray.remote
def a(x):
    x[10] = 5
    return x
    
@ray.remote
def a():
    print('a')

In [5]:
j = [p.remote(x) for x in range(10)]

In [6]:
ray.get(j[0])

In [7]:
ray.get(a.remote())